In [1]:
"""
1、告警内容理解。根据输入的告警信息，结合第三方接口数据，判断当前的异常情况（告警对象、异常模式）；
2、分析方法建议。根据当前告警内容，结合应急预案、运维文档和大语言模型自有知识，形成分析方法的建议；
3、分析内容自动提取。根据用户输入的分析内容需求，调用多种第三方接口获取分析数据，并进行总结；
4、处置方法推荐和执行。根据当前上下文的故障场景理解，结合应急预案和第三方接口，形成推荐处置方案，待用户确认后调用第三方接口进行执行。
"""

'\n1、告警内容理解。根据输入的告警信息，结合第三方接口数据，判断当前的异常情况（告警对象、异常模式）；\n2、分析方法建议。根据当前告警内容，结合应急预案、运维文档和大语言模型自有知识，形成分析方法的建议；\n3、分析内容自动提取。根据用户输入的分析内容需求，调用多种第三方接口获取分析数据，并进行总结；\n4、处置方法推荐和执行。根据当前上下文的故障场景理解，结合应急预案和第三方接口，形成推荐处置方案，待用户确认后调用第三方接口进行执行。\n'

In [2]:
import json
import os
import random
import dashscope
from dashscope.api_entities.dashscope_response import Role
# 从环境变量获取API密钥，避免泄露敏感信息
dashscope.api_key = os.getenv("DASHSCOPE_API_KEY")  # 请设置环境变量
from openai import OpenAI

In [3]:
# 通过第三方接口获取数据库服务器状态
def get_current_status():
    # 生成连接数数据
    connections = random.randint(10, 100)
    # 生成CPU使用率数据
    cpu_usage = round(random.uniform(1, 100), 1)
    # 生成内存使用率数据
    memory_usage = round(random.uniform(10, 100), 1)
    status_info = {
        "连接数": connections,
        "CPU使用率": f"{cpu_usage}%",
        "内存使用率": f"{memory_usage}%"
    }
    return json.dumps(status_info, ensure_ascii=False)

# 封装模型响应函数
def get_response(messages):
    response = dashscope.Generation.call(
        model='qwen-turbo',
        messages=messages,
        tools=tools,
        result_format='message'  # 将输出设置为message形式
    )
    return response
    
current_locals = locals()
current_locals

tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_status",
                "description": "调用监控系统接口，获取当前数据库服务器性能指标，包括：连接数、CPU使用率、内存使用率",
                "parameters": {
                },
                "required": []
            }                
        }
    ]

query = """告警：数据库连接数超过设定阈值
时间：2024-08-03 15:30:00
"""
messages=[
    {"role": "system", "content": "我是运维分析师，用户会告诉我们告警内容。我会基于告警内容，判断当前的异常情况（告警对象、异常模式）"},
    {"role": "user", "content": query}]


In [4]:
while True:
    response = get_response(messages)
    message = response.output.choices[0].message
    messages.append(message)
    #print('response=', response)

    if response.output.choices[0].finish_reason == 'stop':
        break
    
    # 判断用户是否要call function
    if message.tool_calls:
        # 获取fn_name, fn_arguments
        fn_name = message.tool_calls[0]['function']['name']
        fn_arguments = message.tool_calls[0]['function']['arguments']
        arguments_json = json.loads(fn_arguments)
        #print(f'fn_name={fn_name} fn_arguments={fn_arguments}')
        function = current_locals[fn_name]
        tool_response = function(**arguments_json)
        tool_info = {"name": "get_current_weather", "role":"tool", "content": tool_response}
        #print('tool_info=', tool_info)
        messages.append(tool_info)

In [5]:
messages

[{'role': 'system',
  'content': '我是运维分析师，用户会告诉我们告警内容。我会基于告警内容，判断当前的异常情况（告警对象、异常模式）'},
 {'role': 'user', 'content': '告警：数据库连接数超过设定阈值\n时间：2024-08-03 15:30:00\n'},
 Message({'role': 'assistant', 'content': '根据告警内容“数据库连接数超过设定阈值”，我们可以初步判断当前的异常情况可能与数据库的连接数过高有关。为了进一步确认这一情况，我们需要查看当前数据库的实际连接数以及其它相关性能指标如CPU使用率和内存使用率。\n\n接下来，我将调用`get_current_status`函数来获取这些信息。', 'tool_calls': [{'function': {'name': 'get_current_status', 'arguments': '{}'}, 'index': 0, 'id': 'call_3f4d123f60004970bbe340', 'type': 'function'}]}),
 {'name': 'get_current_weather',
  'role': 'tool',
  'content': '{"连接数": 11, "CPU使用率": "100.0%", "内存使用率": "73.6%"}'},
 Message({'role': 'assistant', 'content': '当前数据库的性能指标如下：\n- 连接数：11\n- CPU使用率：100.0%\n- 内存使用率：73.6%\n\n从上述数据可以看出，尽管当前的连接数为11，未直接显示超过阈值，但CPU使用率为100%，这可能是导致高连接数告警的原因。这种情况下，可能存在某些查询或操作占用了大量CPU资源，从而间接导致了连接数的增加。\n\n建议采取以下措施：\n1. 检查是否有长时间运行的查询或任务正在占用CPU资源。\n2. 如果发现有异常的SQL查询，考虑优化查询逻辑或者添加索引来提高查询效率。\n3. 监控数据库的日志文件，查找任何不寻常的行为或错误信息。\n4. 增加数据库服务器的硬件资源（如果可能的话），例如提升CPU或内存容量。\n\n通过这些步骤，应该能